# CSCI 2470 Final Project

## Terms

Let's use `c` to denote constants (e.g., `1` and `2` and `π` and `e`) and `x` to denote variables/parameters (e.g., `x`).

A term/expression `t` is one of the following

* `c`, constants,
* `x`, variable reference,
* `sin(t)`
* `- t`
* `tᵗ`, power
* `t + t`
* `t - t`
* `t × t`

*Note: we denote terms/expressions with `t` rather than `e` because people might misread `e` as the Euler's number.* 

We can size terms by the following function:

```
size(c)       = 1
size(x)       = 1
size(⊖ t)     = 1 + size t
size(t1 ⊕ t2) = 1 + size t1 + size t2
```

We want to make sure that the expected size of a random term `S = E[size(t)]` is bound

So we need to solve the following

|                 constraints                  |
| :------------------------------------------: |
| S = p₀ × 1 + p₁ × (1 + S) + p₂ × (1 + 2 × S) |
|               p₀ + p₁ + p₂ = 1               |
|           pᵢ ≥ 0, ∀ i ∈ {1, 2, 3}            |

which reduces to

|        constraints         |
| :------------------------: |
|  1/S ​≤ p₀ ​≤ (1 + 1/S) / 2  |
|      p₁ = 1-2×p₀+1/S​​       |
|       p₂ = p₀ - 1/S        |

This system is under-constraint. So we will pick `p₀` uniformly randomly.

In [ ]:
class Config:
    def __init__(self, expected_size) -> None:
        assert expected_size >= 1, f"the expected size must be at least 1, given {expected_size}"
        self.expected_size = expected_size

def make_term(config: Config):
    import random
    from sympy import symbols, simplify, expand, factor
    term0_makers = [
        lambda: Constant(random.choice([
            0,
            1,
            2,
            math.pi,
            math.e
            ])),
        lambda: Variable()
    ]
    term1_makers = [
        lambda: Sin(make()),
        lambda: Neg(make()),
    ]
    term2_makers = [
        lambda: Power(make(), make()),
        lambda: Add(make(), make()),
        lambda: Sub(make(), make()),
        lambda: Mul(make(), make()),
    ]
    def make(depth):
        S = config.expected_size
        p0_lower = 1/S
        p0_upper = (1 + 1/S) / 2
        p0 = p0_lower + random.random()*(p0_upper - p0_lower)
        p1 = 1 - 2*p0 + 1/S
        p2 = p0 - 1/S
        assert p0 >= 0, f"p0 is {p0}"
        assert p1 >= 0, f"p1 is {p1}"
        assert p2 >= 0, f"p2 is {p2}"
        assert 0.9 <= p0 + p1 + p2 <= 1.1, f"p0, p1, p2 is {(p0, p1, p2)}; They sum up to {p0 + p1 + p2}"
        makers_by_arity = random.choices(
            [
                term0_makers,
                term1_makers,
                term2_makers
            ],
            weights=[ p0, p1, p2 ],
            k=1
        )[0]
        maker = random.choice(makers_by_arity)
        return maker(depth+1)
    return make(0)

In [182]:
import math
from math import sin, pow

class Term:
    def __str__(self) -> str:
        raise NotImplementedError
    def __repr__(self) -> str:
        return str(self)
    def value(self, x):
        return NotImplementedError
    def size(self):
        return NotImplementedError

class Constant(Term):
    def __init__(self, c) -> None:
        super().__init__()
        self.c = c
    def __str__(self) -> str:
        return str(self.c)
    def value(self, x):
        return self.c
    def size(self):
        return 1

class Variable(Term):
    def __init__(self) -> None:
        super().__init__()
    def __str__(self) -> str:
        return "x"
    def value(self, x):
        return x
    def size(self):
        return 1

class UnaryOperation(Term):
    def __init__(self, t) -> None:
        super().__init__()
        self.t = t
    def f(self, v):
        return NotImplementedError
    def value(self, x):
        return self.f(self.t.value(x))
    def size(self):
        return self.t.size() + 1
class Sin(UnaryOperation):
    def __str__(self) -> str:
        return f"sin({str(self.t)})"
    def f(self, v):
        return sin(v)
class Neg(UnaryOperation):
    def __str__(self) -> str:
        return f"-{str(self.t)}"
    def f(self, v):
        return - v

class BinaryOperation(Term):
    def __init__(self, t1, t2) -> None:
        super().__init__()
        self.t1 = t1
        self.t2 = t2
    def op(self):
        return NotImplementedError
    def __str__(self) -> str:
        return f"({str(self.t1)} {self.op()} {str(self.t2)})"
    def f(self, v1, v2):
        return NotImplementedError
    def value(self, x):
        return self.f(self.t1.value(x), self.t2.value(x))
    def size(self):
        return self.t1.size() + self.t2.size() + 1
class Power(BinaryOperation):
    def op(self):
        return "^"
    def f(self, v1, v2):
        return pow(v1, v2)
class Add(BinaryOperation):
    def op(self):
        return "+"
    def f(self, v1, v2):
        return v1 + v2
class Sub(BinaryOperation):
    def op(self):
        return "-"
    def f(self, v1, v2):
        return v1 - v2
class Mul(BinaryOperation):
    def op(self):
        return "*"
    def f(self, v1, v2):
        return v1 * v2

In practice, we ensure that the constants are reader-friendly.

In [183]:
import random

class Config:
    def __init__(self, expected_size) -> None:
        assert expected_size >= 1, f"the expected size must be at least 1, given {expected_size}"
        self.expected_size = expected_size

def make_term(config: Config):
    term0_makers = [
        lambda depth: Constant(random.choice([
            0,
            1,
            2,
            math.pi,
            math.e
            ])),
        lambda depth: Variable()
    ]
    term1_makers = [
        lambda depth: Sin(make(depth)),
        lambda depth: Neg(make(depth)),
    ]
    term2_makers = [
        lambda depth: Power(make(depth), make(depth)),
        lambda depth: Add(make(depth), make(depth)),
        lambda depth: Sub(make(depth), make(depth)),
        lambda depth: Mul(make(depth), make(depth)),
    ]
    def make(depth):
        S = config.expected_size
        p0_lower = 1/S
        p0_upper = (1 + 1/S) / 2
        p0 = p0_lower + random.random()*(p0_upper - p0_lower)
        p1 = 1 - 2*p0 + 1/S
        p2 = p0 - 1/S
        assert p0 >= 0, f"p0 is {p0}"
        assert p1 >= 0, f"p1 is {p1}"
        assert p2 >= 0, f"p2 is {p2}"
        assert 0.9 <= p0 + p1 + p2 <= 1.1, f"p0, p1, p2 is {(p0, p1, p2)}; They sum up to {p0 + p1 + p2}"
        makers_by_arity = random.choices(
            [
                term0_makers,
                term1_makers,
                term2_makers
            ],
            weights=[ p0, p1, p2 ],
            k=1
        )[0]
        maker = random.choice(makers_by_arity)
        return maker(depth+1)
    return make(0)

In [186]:
t = make_term(Config(10))
str(t)

'(-(1 * sin(sin((sin(-(0 ^ -0)) * -(sin(-(-(--x * 0) ^ -3.141592653589793)) + -sin(x)))))) + ((1 + sin(sin(-(sin((x ^ -x)) - sin(sin(x)))))) * 0))'